### 导包和变量设置

In [5]:
# 引入PyPDFDirectoryLoader，可以从文件夹中一次性加载所有pdf文件
# 然后使用RecursiveCharacterTextSplitter对解析出来的文档进行切分，主要根据分隔符，chunk_size以及overlap等

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers.bm25 import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import os
import gc
import pickle

DOCS_DIR = '/root/autodl-tmp/dataset/rag/A_document'
# DOCS_DIR = '/root/autodl-tmp/dataset/rag/A_small'
EMB_MODEL = '/root/autodl-tmp/models/bge-large-zh-v1_5'
# RERANK_MODEL = "/root/autodl-tmp/models/bge-reranker-large"
RERANK_MODEL = "/root/autodl-tmp/models/jina-reranker-v2-base-multilingual"
PERSIST_DIR = '/root/autodl-tmp/vectorDatabase/faiss_llmsherpa'
QUERY_DIR = '/root/autodl-tmp/dataset/rag/A_question.csv'
SUB_DIR = '/root/autodl-tmp/dataset/rag/submit.csv'
# query = pd.read_csv(QUERY_DIR)

path = "/root/autodl-tmp/dataset/rag/query.pkl"

with open(path, "rb") as f:
    query = pickle.load(f)


sub = pd.read_csv("/root/autodl-tmp/dataset/rag/submit_example.csv")
display(query.head(3))
display(sub.head(3))

/root/miniconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ques_id,question,question_fyde,sub_questions
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,根据2022年年度报告，中国联通在向数字科技领军企业转变的过程中，实现了以下维度的转型升级：...,[根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？...
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,2022年中国联通产业互联网收入同比增长速度为13.8%。,"[告诉我2022年联通产业互联网收入的同比增长速度？, 2022年联通产业互联网收入是多少？]"
2,3,根据2022年度报告，中国联通的企业定位是什么？,根据2022年度报告，中国联通的企业定位是：\n\n1. 成为数字创新服务领导者\n2. 推...,"[根据2022年度报告，中国联通的企业定位是什么？, 中国联通的企业定位包含哪些具体内容？]"


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
2,3,根据2022年度报告，中国联通的企业定位是什么？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."


## 针对query使用子问题拆分方法进行扩充

In [2]:
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 获取上一级目录
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# 将上一级目录添加到 sys.path
sys.path.insert(0, parent_dir)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from typing import List
from glm4.glm4LLM_VLLM import ChatGLM4_LLM

# # LLM Configuration
# def get_subquery_llm(model_name: str = "gpt-4o", temperature: float = 0, max_tokens: int = 4000) -> ChatOpenAI:
#     return ChatOpenAI(temperature=temperature, model_name=model_name, max_tokens=max_tokens)

# Sub-query Decomposition Prompt Template
def create_subquery_decomposition_template() -> PromptTemplate:
    template = """你是一名 AI 助手，任务是将复杂的查询分解为更简单的子查询，以便 RAG 系统处理。
给定原始查询，将其分解为 2 个更简单的子查询，这些子查询结合在一起可以为原始查询提供全面的回答。
请注意，返回结果将两个子查询用‘#’字符拼接起来，不需要输出任何额外的字符

原始查询：{original_query}
示例0:
原始查询：中国联通推出的专属反诈号码是多少？

中国联通推出的专属反诈号码是多少？

示例1:
原始查询：2020年上半年，联通固网宽带的收入和用户数增长了多少？

2020年上半年，联通固网宽带的收入增长了多少？#2020年上半年，联通固网宽带的用户数增长了多少？

示例2:
原始查询：统计数据显示，2022年我国算力规模增长、数字经济和GDP名义分别增长多少？

统计数据显示，2022年我国算力规模增长多少？#统计数据显示，2022年我国数字经济和GDP名义增常多少？

示例3:
原始查询：根据IDC数据，2022年全球数据总产量和过去五年平均增速分别是多少？

根据IDC数据，2022年全球数据总产量和是多少？#2022年过去五年的平均增速分别是多少？
"""
    return PromptTemplate(input_variables=["original_query"], template=template)

# Build Sub-query Decomposition Chain
def build_subquery_decomposer_chain(llm: ChatOpenAI) -> LLMChain:
    prompt_template = create_subquery_decomposition_template()
    return prompt_template | llm

# Function to Decompose Query into Sub-queries
def decompose_query(original_query: str, subquery_chain: LLMChain) -> List[str]:
    response = subquery_chain.invoke(original_query)
    # Parse the sub-queries by splitting lines and removing unwanted text
    sub_queries = [q.strip() for q in response.split('#') if q.strip() and not q.strip().startswith('Sub-queries:')]
    print(sub_queries)
    return sub_queries

def get_sub_queries(query):
    llm = ChatGLM4_LLM(api_base_url="http://localhost:8000/v1")
    subquery_chain = build_subquery_decomposer_chain(llm)
    # original_query = "根据IDC数据，2022年全球数据总产量和过去五年平均增速分别是多少？"
    sub_queries = decompose_query(query, subquery_chain)
    return sub_queries

# print("\nSub-queries:")
# for i, sub_query in enumerate(sub_queries, 1):
#     print(f"{i}. {sub_query}")

In [3]:
query['sub_questions'] = query.apply(get_sub_queries, axis=1)

path = "/root/autodl-tmp/dataset/rag/query.pkl"

with open(path, "wb") as f:
    pickle.dump(query, f)

正在从本地加载模型...
完成本地模型的加载
['根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？', '2022年中国联通在向数字科技领军企业转变的过程中，具体在哪些方面实现了转型升级']
正在从本地加载模型...
完成本地模型的加载
['告诉我2022年联通产业互联网收入的同比增长速度？', '2022年联通产业互联网收入是多少？']
正在从本地加载模型...
完成本地模型的加载
['根据2022年度报告，中国联通的企业定位是什么？', '中国联通的企业定位包含哪些具体内容？']
正在从本地加载模型...
完成本地模型的加载
['2022年联通在大联接业务上取得了什么成果？', '2022年联通在大数据业务上取得了什么成果？']
正在从本地加载模型...
完成本地模型的加载
['2022年上半年，联通5G网络覆盖情况如何？', '2022年上半年，联通精品网络建设总体成果有哪些？']
正在从本地加载模型...
完成本地模型的加载
['2022年半年度报告指出联通在5G网络建设方面取得的进展是什么？', '2022年半年度报告指出联通在物联网应用拓展方面取得的进展是什么？']
正在从本地加载模型...
完成本地模型的加载
['2022年第一季度中国联通应收账款变动主要原因是什么？', '2022年第三季度中国联通应收账款变动主要原因是什么？']
正在从本地加载模型...
完成本地模型的加载
['2022年第一季度，中国联通财务指标中存货变动的主要原因是什么？', '2022年第一季度，中国联通财务指标中存货变动的主要原因可能包括以下几点']
正在从本地加载模型...
完成本地模型的加载
['查询2022年年度报告或季度报告中联通在北京冬奥会和冬残奥会上运用了哪些技术保障通信服务', '2022年年度报告或季度报告中联通在北京冬奥会和冬残奥会上技术保障通信服务的应用效果如何']
正在从本地加载模型...
完成本地模型的加载
['2022年第一季度期末，中国联合网络通信集团有限公司的自有持股比例是多少？', '中国联合网络通信集团有限公司（中国联通）2022年第一季度期末的具体自...']
正在从本地加载模型...
完成本地模型的加载
['介绍2022年联通董事会审计委员会的董事姓名', '2022年联通董事会审计委员会的董事

In [2]:
# 使用 Pandas 的 apply 和 value_counts 方法
length_counts = query['sub_questions'].apply(len).value_counts().to_dict()

print(length_counts)

{2: 100}


In [ ]:
# docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)`

In [6]:
list(query["sub_questions"].values)[0]
# list(query["question"].values)[0]

['根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？',
 '2022年中国联通在向数字科技领军企业转变的过程中，具体在哪些方面实现了转型升级']

In [ ]:
# questions = list(query['question_fyde'].values)

## 针对query使用hyde方法进行扩充

In [ ]:

# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate

# # from yvan_llm_practice.glm4.glm4LLM_VLLM import ChatGLM4_LLM
# # import os
# # import sys
# # # 获取当前工作目录
# # current_dir = os.getcwd()
# # # 获取上一级目录
# # parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# # # 将上一级目录添加到 sys.path
# # sys.path.insert(0, parent_dir)

# from glm4.glm4LLM_VLLM import ChatGLM4_LLM

# # system = """You are an expert about a set of software for building LLM-powered applications called LangChain, LangGraph, LangServe, and LangSmith.

# # LangChain is a Python framework that provides a large set of integrations that can easily be composed to build LLM applications.
# # LangGraph is a Python package built on top of LangChain that makes it easy to build stateful, multi-actor LLM applications.
# # LangServe is a Python package built on top of LangChain that makes it easy to deploy a LangChain application as a REST API.
# # LangSmith is a platform that makes it easy to trace and test LLM applications.

# # Answer the user question as best you can. Answer as though you were writing a tutorial that addressed the user question."""

# system = """你是一名中国联通的专家，精通公司内部的各项业务和技术。你具备以下背景知识：

# 技术前沿：深入了解5G、物联网、大数据和人工智能在通信行业的应用和发展方向。
# 数字化转型：帮助企业和政府客户实现数字化转型，通过智能化解决方案提升效率和竞争力。
# 全球视野：熟悉中国联通在国际市场的布局和合作策略，推动全球通信网络的互联互通。
# 创新驱动：关注技术创新，支持公司在云计算、区块链等新兴领域的探索和应用。
# 用户导向：以用户为中心，致力于提升服务质量和用户体验，满足多样化的客户需求。
# 社会责任：积极参与公益事业，推动教育和环保项目，履行企业社会责任。

# 尽可能好地回答用户问题。回答时要像在写一个教程，以解决用户的问题。
# 一定要注意，直接回答问题，不需要多余的任何冗余"""


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system),
#         ("human", "{question}"),
#     ]
# )
# # llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# llm = ChatGLM4_LLM(api_base_url="http://localhost:8000/v1")
# qa_no_context = prompt | llm | StrOutputParser()

In [ ]:
# question = "2019年底和2020年底，联通宽带端口总数分别为多少？"

# answer = qa_no_context.invoke(
#     {
#         "question": question
#     }
# )
# print(answer)



In [ ]:
# def data_process(question):
#     answer = qa_no_context.invoke(
#         {
#             "question": question
#         }
#     )
#     return answer

# query['question_fyde'] = query.apply(data_process, axis=1)

In [ ]:
# import pickle 

# path = "/root/autodl-tmp/dataset/rag/query.pkl"
# with open(path, "wb") as f:
#     pickle.dump(result, f)

# result["question_fyde"] = result["question_fyde"].apply(lambda x: x.strip())


# with open(path, "rb") as f:
#     result = pickle.load(f)

### PDF文档解析和切分

In [ ]:
# # # 进行数据加载
# # loader = PyPDFDirectoryLoader(DOCS_DIR)

# # 使用 PyPDFDirectoryLoader 加载所有 PDF 文件
# pdf_loader = PyPDFDirectoryLoader(DOCS_DIR)
# documents = pdf_loader.load()

# # 使用 LLMSherpaFileLoader 加载文档
# sherpa_loader = LLMSherpaFileLoader(
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="text",
#     # llmsherpa_api_url="http://127.0.0.1:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true&applyOcr=yes"
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# loaded_documents = [sherpa_loader.load(doc) for doc in documents]

# from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

# loader = LLMSherpaFileLoader(
#     file_path="/root/autodl-tmp/dataset/rag/A_small/AF01.pdf",
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="text",
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# docs = loader.load_and_split(
#     RecursiveCharacterTextSplitter(        
#         chunk_size=200,             
#         chunk_overlap=0,
#         separators = ["。", "！", "？"],
#         keep_separator='end',
#     ),
# )
# # 打印文档数量
# print(len(docs))
# # print(docs[0].page_content)

# # 打印所有第一页的数据出来看下，切分效果如何
# for i, item in enumerate(docs):
#     print(f"the {i} doc's content i: {item.page_content}")

In [6]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader


# 指定 PDF 文件夹路径
pdf_directory = DOCS_DIR

# 获取文件夹中所有 PDF 文件的路径
pdf_files = [os.path.join(pdf_directory, f) for f in os.listdir(pdf_directory) if f.endswith('.pdf')]

# 存储所有切分后的文档块
all_split_documents = []

# 对每个 PDF 文件进行处理
for pdf_file in pdf_files:
    # 使用 LLMSherpaFileLoader 加载文档
    loader = LLMSherpaFileLoader(
        file_path=pdf_file,
        new_indent_parser=True,
        apply_ocr=False,
        strategy="text",
        llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all"
    )
    
    # 使用 RecursiveCharacterTextSplitter 切分文档
    docs_small = loader.load_and_split(
        RecursiveCharacterTextSplitter(
            chunk_size=100,
            chunk_overlap=0,
            separators=["。", "！", "？"],
            # separators=["。"],
            keep_separator='end',
        )
    )

    # docs_big = loader.load_and_split(
    #     RecursiveCharacterTextSplitter(
    #         chunk_size=200,
    #         chunk_overlap=0,
    #         separators=["。", "！", "？"],
    #         # separators=["。"],
    #         keep_separator='end',
    #     )
    # )
    
    # 将切分后的文档块添加到列表中
    all_split_documents.extend(docs_small)

# 去除多余的空格
for doc in all_split_documents:
    # doc.page_content = doc.page_content.replace(" ", "").replace("\n", "")
    # 没有替换\n，因为有些表格，不应该删除换行
    doc.page_content = doc.page_content.replace(" ", "")

# # 输出所有切分后的文档块
# print(all_split_documents)
print("清洗之前的chunk数量：")
len(all_split_documents)

清洗之前的chunk数量：


13986

In [7]:
# 切分出来的效果会又些问题，导致重复等，需要我们进行去重操作
from datasketch import MinHash, MinHashLSH

def get_minhash(doc, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for word in doc.page_content.split():
        m.update(word.encode('utf8'))
    return m

def deduplicate_documents_minhash(documents, threshold=0.8):
    lsh = MinHashLSH(threshold=threshold, num_perm=128)
    unique_docs = []
    minhashes = []

    for i, doc in enumerate(documents):
        m = get_minhash(doc)
        minhashes.append(m)
        if not lsh.query(m):
            lsh.insert(i, m)
            unique_docs.append(doc)

    return unique_docs

# 对已有document进行去重
clean_documents = deduplicate_documents_minhash(all_split_documents)

print("清洗之后的chunk数量：")
print(len(clean_documents))

清洗之后的chunk数量：
6141


In [8]:
for i in range(50):
    print("##########" * 5 + f"{i+1}" + "##########" * 5)
    print(clean_documents[i].page_content)

##################################################1##################################################
中国联通人工智能创新发展论坛在上海成功举办
发布时间：2024年
7
月20日2024年
7
月19日，在中国联通合作伙伴大会期间，成功举办了人工智能创新发展论坛。
##################################################2##################################################
上海市经信委副主任张宏韬、中国联通总经理简勤、GSMA中华区总裁斯寒出席论坛并致辞；中国工程院院士谭建荣，加拿大工程院院士、欧洲科学院院士、香港科技大学教授郭嵩，联通数字科技有限公司总裁、中国联通人工智能创新中心主任朱常波，中国联通人工智能科学家兼人工智能技术总师廉士国，中国联通数字化部副总经理娄瑜发表主旨演讲。
##################################################3##################################################

上海市经信委副主任张宏韬在致辞中表示，中国联通作为中央企业，深入贯彻落实国家“人工智能+”专项行动，在人工智能领域取得了令人瞩目的成就，中国联通人工智能创新中心充分利用中国联通在网、算、云、数、智、端、业的融合优势，推动了人工智能创新应用规模化发展，展现了央企在新时代的责任与担当。
##################################################4##################################################
上海市人民政府也高度重视人工智能的发展，致力于打造开放的创新平台，吸引全球人工智能企业和人才汇聚，共同推动技术交流和国际合作。
##################################################5##################################################
中国联通总经理简勤在致辞中表示，元景2.0不仅是中国联通人工智能技术的升级，更是对人

In [ ]:
# # 使用MinerU进行文档提取(简直无语了，巨慢，8页纸居然用了2min5s左右)
# import os

# from loguru import logger

# from magic_pdf.data.data_reader_writer import FileBasedDataWriter
# from magic_pdf.pipe.UNIPipe import UNIPipe



# try:
#     # current_script_dir = os.path.dirname(os.path.abspath(__file__))
#     # demo_name = 'demo1'
#     # pdf_path = os.path.join(current_script_dir, f'{demo_name}.pdf')

#     current_script_dir = "/root/autodl-tmp/dataset/rag/A_small"
#     demo_name = "AF01"
#     pdf_path = os.path.join(current_script_dir, f'{demo_name}.pdf')
#     pdf_bytes = open(pdf_path, 'rb').read()
#     jso_useful_key = {'_pdf_type': '', 'model_list': []}
#     local_image_dir = os.path.join(current_script_dir, 'images')
#     image_dir = str(os.path.basename(local_image_dir))
#     image_writer = FileBasedDataWriter(local_image_dir)
#     pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
#     # pipe.pipe_classify()
#     # pipe.pipe_analyze()
#     pipe.pipe_parse()
#     md_content = pipe.pipe_mk_markdown(image_dir, drop_mode='none')
#     with open(f'{demo_name}.md', 'w', encoding='utf-8') as f:
#         f.write(md_content)
# except Exception as e:
#     logger.exception(e)


In [ ]:
# # 使用llmsherpa
# # from llmsherpa.readers import LayoutPDFReader

# # llmsherpa_api_url = "http://127.0.0.1:5001//api/parseDocument?renderFormat=all"
# # pdf_url = "/root/autodl-tmp/dataset/rag/A_small/AF01.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
# # pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# # doc = pdf_reader.read_pdf(pdf_url)

# from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

# loader = LLMSherpaFileLoader(
#     file_path="/root/autodl-tmp/dataset/rag/A_small/AF01.pdf",
#     new_indent_parser=True,
#     apply_ocr=False,
#     strategy="html",
#     # llmsherpa_api_url="http://127.0.0.1:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true&applyOcr=yes"
#     llmsherpa_api_url="http://0.0.0.0:5001/api/parseDocument?renderFormat=all",
# )

# docs = loader.load()




In [39]:
# # 如果index不存在，创建一个index

# from pinecone import Pinecone, ServerlessSpec

# from langchain_pinecone import PineconeVectorStore

# embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)

# api_key = "pcsk_43QZGm_EEk7V2hAogimUuXn7uW9xVdf8UBaHDaWt5mjCbE5AYAtwxaqhThPFfkK42FpLP"
# os.environ['PINECONE_API_KEY'] = api_key


# pc = Pinecone(api_key=api_key)

# import time

# index_name = "langchain-index"  # change if desired

# existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# if index_name not in existing_indexes:
#     pc.create_index(
#         name=index_name,
#         dimension=1024,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#     while not pc.describe_index(index_name).status["ready"]:
#         time.sleep(1)

# # index = pc.Index(index_name)

# vectorstore = PineconeVectorStore.from_existing_index(index_name, embedding=embeddings)

### 文本块向量化（比赛限定使用bge-large-zh-v1.5模型）

In [9]:
# 只有当重新写入的时候才需要运行

embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)

#使用faiss作为数据库
vectordb = FAISS.from_documents(   
    documents=clean_documents,
    embedding=embeddings,
)

faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

vectordb.save_local(PERSIST_DIR)

# from langchain_pinecone import PineconeVectorStore

# index_name = "langchain-index"  # change if desired

# os.environ['PINECONE_API_KEY'] = "pcsk_43QZGm_EEk7V2hAogimUuXn7uW9xVdf8UBaHDaWt5mjCbE5AYAtwxaqhThPFfkK42FpLP"

# # vector_store = PineconeVectorStore(index=index, embedding=embeddings)
# # 下面这种生成方式可以更加快速地写入数据，1w4数据，写入只需要2min
# vectorstore = PineconeVectorStore.from_documents(clean_documents, index_name=index_name, embedding=embeddings, batch_size=100, pool_threads=10)



/tmp/ipykernel_5403/1354659199.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)
Batches: 100%|██████████| 192/192 [00:24<00:00,  7.70it/s]


In [ ]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold",
#     search_kwargs={"k": 1, "score_threshold": 0.5},
# )

### 混合检索器

#### bm25 
- k1 较高的 k1 值意味着词频对评分的影响更大。
- b  当 b=1 时，文档长度的影响最大；当b = 0 时，文档长度不影响评分。
- langchain 默认切分英文split()，中文需要jieba分词

In [10]:
import jieba
dense_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# dense_retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold",
#     search_kwargs={"k": 5, "score_threshold": 0.5},
# )


bm25_retriever = BM25Retriever.from_documents(
    all_split_documents, 
    k=5, 
    bm25_params={"k1": 1.5, "b": 0.75}, 
    preprocess_func=jieba.lcut
)
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, dense_retriever], weights=[0.5, 0.5])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.659 seconds.
Prefix dict has been built successfully.


### 文本召回和重排

In [ ]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import CrossEncoderReranker
# from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# def rerank(questions, retriever, top_n=5, cut_len=384):
#     rerank_model = HuggingFaceCrossEncoder(model_name=RERANK_MODEL)
#     compressor = CrossEncoderReranker(model=rerank_model, top_n=top_n)
#     compression_retriever = ContextualCompressionRetriever(
#         base_compressor=compressor, base_retriever=retriever
#     )
#     rerank_answers = []
#     for question in tqdm(questions):
#         relevant_docs = compression_retriever.invoke(question)
#         answer=''
#         for rd in relevant_docs:
#             answer += rd.page_content
#         rerank_answers.append(answer[:cut_len])
#     return rerank_answers

# questions = list(query['question'].values)
# rerank_answers = rerank(questions, ensemble_retriever)
# print(rerank_answers[0])


In [11]:
from concurrent.futures import ThreadPoolExecutor
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import ContextualCompressionRetriever
from tqdm import tqdm

from datasketch import MinHash, MinHashLSH

def get_minhash(doc, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for word in doc.page_content.replace(" ", "").split():
        m.update(word.encode('utf8'))
    return m

def deduplicate_documents_minhash(documents, threshold=0.8):
    lsh = MinHashLSH(threshold=threshold, num_perm=128)
    unique_docs = []
    minhashes = []

    for i, doc in enumerate(documents):
        m = get_minhash(doc)
        minhashes.append(m)
        if not lsh.query(m):
            lsh.insert(i, m)
            unique_docs.append(doc)

    return unique_docs

def retrieve_documents(query, retriever):
    docs = retriever.invoke(query)
    # print(f"the docs is: {docs}")
    return docs

def rerank_documents(query, documents, rerank_model, top_n=5):
    compressor = CrossEncoderReranker(model=rerank_model, top_n=top_n)
    relevant_docs = compressor.compress_documents(documents, query)
    return relevant_docs


def rerank(questions, sub_questions, questions_fyde, retriever, rerank_model_name, top_n=3, cut_len=384):
    rerank_model = HuggingFaceCrossEncoder(model_name=rerank_model_name)
    rerank_answers = []

    for question, sub_question, question_fyde in tqdm(zip(questions, sub_questions, questions_fyde)):
        # 单次调用召回
        docs_quer_origin = retrieve_documents(question, retriever)
        
        # doc_query_new = []
        # for sub in sub_question:
        #     doc_query = retrieve_documents(sub, retriever)
        # doc_query_new.extend(doc_query)
        docs_query_first = retrieve_documents(sub_question[0], retriever)
        docs_query_second = retrieve_documents(sub_question[1], retriever)
        docs_fyde = retrieve_documents(question_fyde, retriever)

        # 合并文档
        all_docs = docs_quer_origin + docs_query_first + docs_query_second + docs_fyde

        # 去重文档
        unique_docs = deduplicate_documents_minhash(all_docs)

        # print(unique_docs)

        # 重新排序
        reranked_docs = rerank_documents(question, unique_docs, rerank_model, top_n)

        # 提取内容
        answer = '\n'.join(doc.page_content for doc in reranked_docs)
        rerank_answers.append(answer[:cut_len])

    return rerank_answers


In [ ]:
# 使用示例
questions = list(query["question"].values)
sub_questions = list(query['sub_questions'].values)
questions_fyde = list(query["question_fyde"].values)

rerank_answers = rerank(questions, sub_questions, questions_fyde, ensemble_retriever, RERANK_MODEL)
print(rerank_answers[0])

### 提交

In [16]:
def emb(answers, emb_batch_size = 4):
    model = SentenceTransformer(EMB_MODEL, trust_remote_code=True).half()
    all_sentence_embeddings = []
    for i in tqdm(range(0, len(answers), emb_batch_size), desc="embedding sentences"):
        batch_sentences = answers[i:i+emb_batch_size]
        sentence_embeddings = model.encode(batch_sentences, normalize_embeddings=True)
        all_sentence_embeddings.append(sentence_embeddings)
    all_sentence_embeddings = np.concatenate(all_sentence_embeddings, axis=0)
    print('emb_model max_seq_length: ', model.max_seq_length)
    print('emb_model embeddings_shape: ', all_sentence_embeddings.shape[-1])
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return all_sentence_embeddings

all_sentence_embeddings = emb(rerank_answers)
sub['answer'] = rerank_answers
sub['embedding']= [','.join([str(a) for a in all_sentence_embeddings[i]]) for i in range(len(all_sentence_embeddings))]
sub.to_csv(SUB_DIR, index=None)
sub.head()

embedding sentences: 100%|██████████| 25/25 [00:00<00:00, 33.90it/s]


emb_model max_seq_length:  512
emb_model embeddings_shape:  1024


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,\n我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加 速...,"-0.02802,-0.006622,-0.01736,0.004593,0.01402,0..."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,公司产业互联网继续按下快进键，2022年收入首破700亿大关，同比增长达到29%，实现规模、...,"-0.02975,-0.006638,-0.01276,0.003176,0.0429,0...."
2,3,根据2022年度报告，中国联通的企业定位是什么？,\n公司基本情况\n1公司简介3公司主要会计数据和财务指标\n||公司股票简况\n|---|...,"-0.04153,-0.01656,-0.04892,0.014305,0.03244,0...."
3,4,2022年联通在“大联接”和“大数据”业务上取得了什么成果？,在数字政府、数字金融、智慧文旅、数据安全等领域，实现省市级标杆项目规模复制。联通大数据业务保...,"-0.04303,-0.02644,-0.04166,0.0067,0.0238,0.057..."
4,5,2022年上半年，联通在精品网络建设上有什么成果？,公司固网宽带业务延续了去年高速增长的良好态势，上半年实现宽带接入收入 230 亿元，同比提...,"-0.01903,-0.0219,-0.04358,-0.01787,0.005768,0...."


In [ ]:
import time
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5])

# question = "2022年上半年，联通在精品网络建设上有什么成果？"
# question = "根据2022年度报告，中国联通的企业定位是什么？"
# question = "2022年联通在“大联接”和“大数据”业务上取得了什么成果？"
# question = "根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？"

question = "根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？"

start = time.time()
docs_query_first = retrieve_documents(question, ensemble_retriever)

# docs_query_first = retrieve_documents(question, bm25_retriever)

# docs_query_first = retrieve_documents(question, dense_retriever)

# docs_query_first = retrieve_documents(question, faiss_retriever)

end = time.time()

print(f"the retrieve cost time: {end - start}s")
rerank_model = HuggingFaceCrossEncoder(model_name=RERANK_MODEL, model_kwargs = {'device': 'cuda', 'trust_remote_code': True})



Batches: 100%|██████████| 1/1 [00:00<00:00, 82.44it/s]


the retrieve cost time: 0.028432846069335938s


In [28]:
docs_query_first

[Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AF19.pdf'}, page_content='“请问谁知道在空中飞着的是什么东西呢？”志愿者们以老师的身份向桂林阳朔杨堤乡凤凰小学的学生们提问。课堂气氛异常热烈，同学们积极举手回答问题，脸上洋溢着好奇与兴奋。'),
 Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AT12.pdf'}, page_content='中国联通将继续围绕国家所需、产业链供应链所困，以战略性新兴产业和未来产业为主线，全力成为国家战略科技力量的重要组成部分。'),
 Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY02.pdf'}, page_content='2022年是中国联通全面贯彻落实新战略规划的起跑之年。'),
 Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY01.pdf'}, page_content='中国联合网络通信股份有限公司2022年年度报告摘要\n第一节重要提示\n1本年度报告摘要来自年度报告全文，为全面了解本公司的经营成果、财务状况及未来发展规划，投资者应当到www.sse.com.cn；www.chinaunicom-a.com网站仔细阅读年度报告全文。'),
 Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AT10.pdf'}, page_content='2023年，中国联通充分发挥在建设现代化产业体系、构建新发展格局中的科技创新、产业控制、安全支撑作用，全力拓展联网通信、算网数智两类主营业务，主营收入稳步增长，用户规模稳步扩大，业务结构显著优化，盈利水平显著提升。'),
 Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY11.pdf'}, pag

In [ ]:
class Reranker:
    def rerank(self, query, passages, top_k=32768):
        raise NotImplementedError


from sentence_transformers import CrossEncoder

class CrossEncoderReranker(Reranker):
    def __init__(self):
        self.reranker_model = CrossEncoder(RERANK_MODEL, max_length=512, device="cuda", automodel_args={"torch_dtype": torch.float16})
        
    def rerank(self, query, passages, top_k=5):
        score_inputs = [[query, passage.page_content] for passage in passages]
        scores = self.reranker_model.predict(score_inputs)
        # result = [{'question': passage, 'score': score} for passage, score in zip(passages, scores)]
        result = [{'idx': idx, 'question': passage, 'score': score} for idx, (passage, score) in enumerate(zip(passages, scores))]
        sorted_result = sorted(result, key=lambda x: x['score'], reverse=True)
        return sorted_result[:top_k]

In [ ]:
rerank_model = CrossEncoderReranker()

rerank_model.rerank(question, docs_query_first, 5)


[{'idx': 2,
  'question': Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY02.pdf'}, page_content=' 公司固网宽带业务延续了去年高速增长的良好态势，上半年实现宽带接入收入 230 亿元，同比提升 适度加大战略投入，基础网络能力大幅提升 中国联通始终坚持网络在企业发展中的基础地位，适度加大战略投入，上半年的精品网络建设卓 有成效，为公司有根生长筑牢发展底座。'),
  'score': 0.9770508},
 {'idx': 0,
  'question': Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY02.pdf'}, page_content='公司固网宽带业务延续了去年高速增长的良好态势，上半年实现宽带接入收入230亿元，同比提升适度加大战略投入，基础网络能力大幅提升中国联通始终坚持网络在企业发展中的基础地位，适度加大战略投入，上半年的精品网络建设卓有成效，为公司有根生长筑牢发展底座。'),
  'score': 0.9663086},
 {'idx': 3,
  'question': Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY02.pdf'}, page_content='5G/4G精品网建设方面，已经实现重点乡镇以上场景室外连续覆盖，5G中频规模和覆盖水平与行业相当。'),
  'score': 0.91796875},
 {'idx': 5,
  'question': Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY07.pdf'}, page_content='上半年，公司聚焦地区网络质量和客户感知持续提升，移网和固网宽带实时满意度持续提升，网络时延指标行业最优。'),
  'score': 0.90722656},
 {'idx': 7,
  'question': Document(metadata={'sou

In [ ]:
# rerank_model.reranker_model.model.device

device(type='cuda', index=0)

In [33]:
start = time.time()
reranked_docs = rerank_documents(question, docs_query_first, rerank_model , 5)
end = time.time()

print(f"the rerank cost time: {end - start}s")

the rerank cost time: 0.1523590087890625s


In [19]:
docs_query_first

[Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY12.pdf'}, page_content='2019年上半年，创新业务成为稳定公司收入的主要驱动力。产业互联网业务收入同比增长43%，达到人民币167亿元，占整体主营业务收入比例提高至13%。'),
 Document(id='24425f68-1c8d-469d-93bc-3cbe40270591', metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY03.pdf'}, page_content='联通云加速发展，实现收入3人民币268.7亿元，同比提升142.0%；IDC实现收入人民币186.1亿元，同比提升12.9%。'),
 Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AZ09.pdf'}, page_content='算力作为数字经济核心产业的重要底座支撑，对上游软硬件产业的拉动作用日渐凸显，2022年全国电子信息制造业实现营业收入15.4万亿元，同比增长5.5%。'),
 Document(id='b18ff517-01a9-4082-887e-fe501a6c166b', metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY01.pdf'}, page_content='物联网实现收入86亿元，同比增长达到42%，非连接收入增长63%，增速大幅领跑行业。大计算积厚成势，“联通云”继续翻倍增长，2022年实现收入361亿元，同比增速达到121%。'),
 Document(metadata={'source': '/root/autodl-tmp/dataset/rag/A_document/AY11.pdf'}, page_content='2019年，公司创新业务保持快速发展，产业互联网收入同比增长43%，达到人民币329亿元，占整体主营业务收入比例达到12.4%，成为稳定主营业务收入的重要驱动力。'),
 Document(id='18ef4b27-3e78-

In [17]:
for i in range(40,60):
    print("##########"* 5, f"{i+1}", "##########" * 5)
    print(rerank_answers[i])


################################################## 41 ##################################################
郭华对工作的严谨态度和高度的责任心，使得他能够在每一次重大任务面前，从容不迫，
确保任务顺利完成。为了保障通信业务的顺利进行，郭华常常放弃个人休息时间，全身心地投入到工作中去。
郭华以他的实际行动证明了，只要有责任感、有担当，就能在平凡的岗位上创造出不平凡的业绩，为公司的发展和国家的重要活动提供坚实的保障。郭华的领导风格也为团队注入了强大的凝聚力和战斗力。
他常年带领团队奔波于北京、广州、杭州和西安，这些城市不仅地理位置各异，而且在项目启动初期交通极为不便。面对这样的挑战，郭年毫不退缩，他经常顶着40多度的酷暑，徒步数公里到达项目现场。
################################################## 42 ##################################################
本次论坛以“同行跃新智联未来”为主题，重磅发布“知驭”“知略”“知途”三大车联网创新产品和联通人车家生活全生态平台。
大会以“算网筑基拥抱智能共促生态发展”为主题，全面展示了中国联通在智算领域全新升级的技术内核及产品能力，并联合行业生态合作伙伴及行业专家，围绕算网数智、人工智能等前沿科技与热点话题展开讨论与分享。
数智县域向新无界——中国联通人工智能赋能全域数字化论坛在沪召开
发布时间：2024年
7
月20日
7
月19日，2024中国联通合作伙伴大会期间，以“数智县域向新无界”为主题的人工智能赋能全域数字化论坛顺利召开。
################################################## 43 ##################################################
9间接带动经济总产出和经济增加值的增长率测算方法同上。315亿元。二是带动各行业企业对数字化转型的投资。5G、10项目签约金额达209亿元，同比增长28%。
2市场规模为2023年全年数据统计，主要依据企业财报、人员访谈、可信云评估、历史数据等得出。对于市场数据不明确的领域，只发布头部企业整体

In [ ]:
query["sub_questions"][3]

In [ ]:
from langchain_core.runnables import RunnablePassthrough

hyde_chain = RunnablePassthrough.assign(hypothetical_document=qa_no_context)

hyde_chain.invoke(
    {
        "question": "how to use multi-modal models in a chain and turn chain into a rest api"
    }
)

### 后续可能提分点
- 引入LLM
   * LLM 递归判断/抽取
   * rag-fusion 查询改写
   * 构建知识图谱



### 注意：
- 在分块、重排等过程中可以使用公开库和模型，但禁止使用LLM直接生成最终答案。
- 禁止使用LLM继续调整精排得到的文本块，如压缩文本块长度；
- 禁止使用LLM直接从文档获取问题答案。